In [1]:
import pandas as pd
import sklearn

import service
from classification.OneRClassifier import OneRClassifier
from classification.NaiveBayesClassifier import NaiveBayesClassifier

In [2]:
classifiers = [OneRClassifier()]  # TODO

#### Check classifiers (scikit-learn)

In [3]:
import warnings
from sklearn.utils.estimator_checks import estimator_checks_generator

for clf in classifiers:
    for (estimator, check) in estimator_checks_generator(clf):
        try:
            check(estimator)
        except KeyError as e:
            if "encountered an unknown value" in str(e):
                continue
            warnings.warn(str(e))

#### data_4x

In [4]:
df = pd.read_csv('data/data_4x.csv')
df.head()

,x0,x1,x2,x3,y
0,0,0,1,0,0
1,1,0,0,1,1
2,2,0,1,0,1
3,0,1,0,0,1
4,0,1,1,0,1


In [5]:
X, y = df.drop(columns='y'), df['y']
X.shape, y.shape

((10, 4), (10,))

In [6]:
y.replace(0, 'y0', inplace=True)
y.replace(1, 'y1', inplace=True)
y = list(y)

In [7]:
X_pred = pd.DataFrame({'x0': [2], 'x1': [1], 'x2': [1], 'x3': [1]})

##### OneRClassifier

In [8]:
clf = OneRClassifier()
clf.fit(X, y);

In [9]:
service.OneRClassifier_info(clf)

x[3] == 0: y = y1
x[3] == 1: y = y0


In [10]:
clf.predict(X_pred)

array(['y0'], dtype='<U2')

Compare with similar scikit-learn model

In [11]:
sk_clf = sklearn.tree.DecisionTreeClassifier(max_depth=1)
sk_clf.fit(X, y);

In [12]:
service.sklearn_DecisionTreeClassifier_info(sk_clf)

x[3] <= 0.5: y = y1
x[3] > 0.5: y = y0


In [13]:
sk_clf.predict(X_pred)

array(['y0'], dtype='<U2')

##### NaiveBayesClassifier

In [14]:
clf = NaiveBayesClassifier()
clf.fit(X, y);

[y  y   
0  0  0    0.571429
      1    0.285714
1  1  2    0.444444
      0    0.333333
      1    0.222222
Name: attr, dtype: float64, 0  y  0
0  0  0    0.428571
   1  0    0.571429
1  0  1    0.428571
   1  1    0.571429
Name: attr, dtype: float64, y  y   
0  0  1    0.666667
      0    0.333333
1  1  0    0.500000
      1    0.500000
Name: attr, dtype: float64, y  y   
0  0  1    0.666667
      0    0.333333
1  1  0    0.750000
      1    0.250000
Name: attr, dtype: float64]
[0.3333333333333333, 0.5, 0.5, 0.5]


In [15]:
clf.predict(pd.DataFrame({'x0': [2], 'x1': [1], 'x2': [1], 'x3': [1]}))

array(Ellipsis, dtype=object)

##### DecisionTreeClassifier

In [16]:
# TODO

##### KNeighborsClassifier

In [17]:
# TODO

#### data_3x

In [18]:
df = pd.read_csv('data/data_3x.csv')
df.head()

,dataset,x0,x1,x2,y
0,1,0,1,2,1
1,1,1,0,1,1
2,1,0,1,1,0
3,1,0,0,1,1
4,1,0,0,2,1


In [19]:
res_df = pd.DataFrame(
    index=[str(clf) for clf in classifiers],
    columns=df['dataset'].unique()
)

X_pred = pd.DataFrame({'x0': [1], 'x1': [1], 'x2': [1]})
for clf in classifiers:
    for df_i, df_v in df.groupby('dataset'):
        df_v = df_v.drop(columns=['dataset'])
        X, y = df_v.drop(columns='y'), df_v['y']
        clf.fit(X, y)
        y_pred = clf.predict(X_pred)[0]
        res_df.loc[str(clf), df_i] = y_pred

res_df

,1,2,3,4
OneRClassifier(),1,1,1,1
